## Preprocess the Sim2Real dataset

The Sim2Real dataset uses jpeg images to represent wildfire masks. This representation is very lightweight but slow to work with. 
To benchmark faster, we convert them to binary NumPy files. These files are ~100 times heavier and it requires an overhead time to convert from jpeg to them, but once they are created and stored, they allow 50--100x faster operations.

1. Download the dataset from [Sim2Real-Fire GitHub repository](https://github.com/TJU-IDVLab/Sim2Real-Fire).
2. Extract all files from the dataset into your `Dataset` folder
3. Run the preprocessing function to convert from the jpeg representation to the NumPy one, and compute burn maps

In [9]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import requred modules
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add code to path
module_path = os.path.abspath(".") + "/code"
if module_path not in sys.path:
    sys.path.append(module_path)


from dataset import preprocess_sim2real_dataset, load_scenario_npy, compute_and_save_burn_maps_sim2real_dataset
from wrappers import wrap_log_sensor_strategy, wrap_log_drone_strategy
from Strategy import RandomDroneRoutingStrategy, return_no_custom_parameters, SensorPlacementOptimization, RandomSensorPlacementStrategy, LoggedOptimizationSensorPlacementStrategy,DroneRoutingOptimizationSlow, DroneRoutingOptimizationModelReuse, DroneRoutingOptimizationModelReuseIndex, LoggedDroneRoutingStrategy, LogWrapperDrone, LogWrapperSensor
from benchmark import run_benchmark_scenario,run_benchmark_scenarii_sequential, get_burnmap_parameters,run_benchmark_scenarii_sequential_precompute, benchmark_on_sim2real_dataset_precompute
from displays import create_scenario_video
from new_clustering import get_wrapped_strategy

Uncomment the cell below to preprocess the dataset.
- `n_max_scenarii_per_layout` controls the number of scenarios we convert from jpeg to NumPy files for each layout.
- If executed on all ~50 layouts of the dataset, the code below takes ~15mins to run and generates **400 GB** of data. Make sure to have the space.

In [5]:
# preprocess_sim2real_dataset("MinimalDataset/", n_max_scenarii_per_layout=100) 

Converting JPG scenarios to NPY for MinimalDataset/
Computing burn maps...
Computing burn map for MinimalDataset/0002/scenarii/ and files with extension .npy


100%|██████████| 5/5 [00:00<00:00, 31.16it/s]


Computing burn map for MinimalDataset/0001/scenarii/ and files with extension .npy


100%|██████████| 5/5 [00:00<00:00, 50.21it/s]


We can now run the benchmark function on any scenario.
1. We load the scenario using `load_scenario_npy` since it is in preprocessed `npy` format
2. Werun the benchmark using `run_benchmark_scenario` that takes as input:
    - The `scenario`, 
    - The sensor placement strategy and the drone routing strategy
    - A dictionary `custom_initialization_parameters` that contains any custom initialization inputs for your strategy functions (such as the burn map if your strategy needs a burn map as input)
    - A python function `custom_step_parameters_function` that returns a dictionary of custom inputs for your routing function. This function is executed by the benchmarking code at each time step. Your routing function will be called with: `routing(custom_step_parameters_function())` internally

In [3]:
# change values here to change benchmarking parameters
def my_automatic_layout_parameters(scenario:np.ndarray):
    return {
        "N": scenario.shape[1],
        "M": scenario.shape[2],
        "max_battery_distance": -10,
        "max_battery_time": 20,
        "n_drones": 10,
        "n_ground_stations": 1,
        "n_charging_stations": 5,
    }

In [4]:
# That's very fast to run!
print("starting benchmark")
time_start = time.time()
scenario = load_scenario_npy("MinimalDataset/0001/scenarii/0001_00002.npy")
results, (position_history, ground, charging)  = run_benchmark_scenario(scenario, SensorPlacementOptimization, DroneRoutingOptimizationModelReuseIndex, custom_initialization_parameters = {"burnmap_filename": "./MinimalDataset/0001/burn_map.npy", "load_from_logfile": False, "reevaluation_step": 5, "optimization_horizon":5, "strategy_drone": DroneRoutingOptimizationModelReuseIndex, "strategy_sensor": RandomSensorPlacementStrategy}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
print(results)
print(f"Time taken to run benchmark on the scenario: {time.time() - time_start} seconds")




starting benchmark
calling julia optimization model
NEW STRATEGY 2
Set parameter Username
Set parameter LicenseID to value 2612529
Academic license - for non-commercial use only - expires 2026-01-21
Took 43.11301804194227 seconds
Average risk equals
optimization finished
Charging Station Locations from Julia Optimization Model:  []
ground sensor locations
[(105, 229)]
charging station locations
[(109, 181), (110, 230), (111, 176), (114, 181), (114, 186)]
Creating initial routing model (reusable)
--- parameters for julia (Julia indexing) ---
burnmap_filename: ./MinimalDataset/0001/burn_map.npy
n_drones: 10
charging_stations_locations: [(111, 231), (115, 182), (112, 177), (115, 187), (110, 182)]
ground_sensor_locations: [(106, 230)]
optimization_horizon: 5
Set parameter Username
Set parameter LicenseID to value 2612529
Academic license - for non-commercial use only - expires 2026-01-21
Model created in 7.1021349998191 seconds
DEBUG: Charging Stations received in Julia:
[(111, 231), (115,

In [5]:
print("working on it...")
def custom_initialization_parameters_function(input_dir: str, layout_name: str = None):
    print(f"input_dir: {input_dir}")
    burnmap_path = f"{'/'.join(input_dir.strip('/').split('/')[:-1])}/burn_map.npy"
    os.makedirs("logs", exist_ok=True)

    return {
        "burnmap_filename": burnmap_path,
        "reevaluation_step": 5,
        "optimization_horizon": 10,
        "strategy_drone": DroneRoutingOptimizationModelReuseIndex,
        "strategy_sensor": RandomSensorPlacementStrategy,
        "recompute_logfile": False  # toggle this to force refresh
    }

clustered_strategy = get_wrapped_strategy(DroneRoutingOptimizationModelReuseIndex)

# change values here to change benchmarking parameters

simulation_parameters =  {
    "max_battery_distance": -1,
    "max_battery_time": 20,
    "n_drones": 10,
    "n_ground_stations": 1,
    "n_charging_stations": 5,
}

# Run the benchmark and collect metrics
print("Running benchmarks...")
metrics_by_layout = benchmark_on_sim2real_dataset_precompute(
    dataset_folder_name="MinimalDataset/",
    ground_placement_strategy=RandomSensorPlacementStrategy,
    drone_routing_strategy=clustered_strategy,
    custom_initialization_parameters_function=custom_initialization_parameters_function,
    custom_step_parameters_function=return_no_custom_parameters,
    max_n_scenarii=1,
    starting_time=0,
    simulation_parameters=simulation_parameters
)

working on it...
Running benchmarks...
simulation_parameters:  {}

 --- 
 Processing layout MinimalDataset/0002
other case : automatic_initialization_parameters:  {'N': 205, 'M': 229, 'max_battery_distance': -1, 'max_battery_time': 20, 'n_drones': 5, 'n_ground_stations': 10, 'n_charging_stations': 5, 'input_dir': 'MinimalDataset/0002/scenarii/'}
input_dir: MinimalDataset/0002/scenarii/
RandomSensorPlacementStrategy
[init] Number of clusters: 5
  Cluster 0: [(19, 182)]
  Cluster 1: [(166, 35)]
  Cluster 2: [(171, 97)]
  Cluster 3: [(183, 164)]
  Cluster 4: [(187, 225)]

🚀 Running cluster 0 with 1 charging stations and 1 drones
  🧱 Bounding grid: 21 x 21, origin: (9, 172)
  ✅ Strategy initialized for cluster 0

🚀 Running cluster 1 with 1 charging stations and 1 drones
  🧱 Bounding grid: 21 x 21, origin: (156, 25)
  ✅ Strategy initialized for cluster 1

🚀 Running cluster 2 with 1 charging stations and 1 drones
  🧱 Bounding grid: 21 x 21, origin: (161, 87)
  ✅ Strategy initialized for clus

JuliaError: BoundsError: attempt to access 1-element PyList{Any} at index [2]
Stacktrace:
  [1] throw_boundserror(A::PyList{Any}, I::Tuple{Int64})
    @ Base ./essentials.jl:14
  [2] checkbounds
    @ ./abstractarray.jl:699 [inlined]
  [3] getindex
    @ ~/.julia/packages/PythonCall/WMWY0/src/Wrap/PyList.jl:32 [inlined]
  [4] macro expansion
    @ ~/.julia/packages/MutableArithmetics/bmgkg/src/rewrite.jl:371 [inlined]
  [5] macro expansion
    @ ~/.julia/packages/JuMP/CU7H5/src/macros.jl:264 [inlined]
  [6] macro expansion
    @ ~/.julia/packages/JuMP/CU7H5/src/macros/@constraint.jl:171 [inlined]
  [7] macro expansion
    @ ~/.julia/packages/JuMP/CU7H5/src/macros.jl:400 [inlined]
  [8] solve_index_next_move_routing(routing_model::IndexRoutingModel, reevaluation_step::Int64, drone_locations::PyList{Any}, drone_states::PyList{Any}, battery_level::PyList{Any})
    @ Main ~/Desktop/Climate/wildfire_drone_routing/julia/drone_routing_opt.jl:1061
  [9] pyjlany_call(self::typeof(solve_index_ne

movement_plan: [[("fly", (167, 36)), ("fly", (167, 36)), ("fly", (20, 183)), ("fly", (20, 183)), ("fly", (184, 165))], [("fly", (166, 36)), ("fly", (166, 36)), ("fly", (20, 183)), ("fly", (20, 183)), ("fly", (185, 164))], [("fly", (166, 36)), ("fly", (166, 36)), ("fly", (20, 183)), ("fly", (20, 183)), ("fly", (185, 164))], [("fly", (166, 36)), ("fly", (166, 36)), ("fly", (20, 183)), ("fly", (20, 183)), ("fly", (185, 164))], [("fly", (166, 36)), ("fly", (166, 36)), ("fly", (20, 183)), ("fly", (20, 183)), ("fly", (185, 164))]]


In [8]:
from benchmark import benchmark_on_sim2real_dataset_precompute, build_custom_init_params, return_no_custom_parameters
from Strategy import RandomSensorPlacementStrategy, DroneRoutingOptimizationExample
from plot_metrics import plot_all_metrics_across_layouts
from plot_violin import gather_data_from_layouts, plot_violin_for_each_metric

# Create output directories
os.makedirs("plots", exist_ok=True)
os.makedirs("plots_violin", exist_ok=True)

# --- Step 2: Generate line plots per layout ---
plot_all_metrics_across_layouts(
    root_folder="MinimalDataset",
    sensor_strategy_cls=RandomSensorPlacementStrategy,
    drone_strategy_cls=DroneRoutingOptimizationExample,
    max_n_scenarii=100,
    starting_time=0
)

# --- Step 3: Generate violin plots across strategies (optional but useful for comparison) ---
from Strategy import RandomDroneRoutingStrategy, LoggedSensorPlacementStrategy, LoggedDroneRoutingStrategy

STRATEGIES = [
    ("Random", RandomSensorPlacementStrategy, RandomDroneRoutingStrategy),
    ("Logged", LoggedSensorPlacementStrategy, LoggedDroneRoutingStrategy),
    ("Optimized", RandomSensorPlacementStrategy, DroneRoutingOptimizationExample)
]

df_all = gather_data_from_layouts(
    root_folder="MinimalDataset",
    strategies=STRATEGIES,
    custom_init_params_fn=build_custom_init_params,
    custom_step_params_fn=return_no_custom_parameters,
    starting_time=0,
    max_n_scenarii=100
)

if not df_all.empty:
    plot_violin_for_each_metric(df_all, output_dir="plots_violin")

ImportError: cannot import name 'DroneRoutingOptimizationExample' from 'Strategy' (/Users/puech/Desktop/Climate/wildfire_drone_routing/code/Strategy.py)

In [ ]:
create_scenario_video(scenario[:len(position_history)],drone_locations_history=position_history,starting_time=0,out_filename='test_simulation', ground_sensor_locations = ground, charging_stations_locations = charging)

We can visualize the strategy in action by creating a video of the scenario. 
1. Use the `return_history` parameter or `run_benchmark_scenario` to output the log of ground sensor and drone positions during the benchmark
2. Use the `create_scenario_video` function to compile the video. This can take a couple seconds

In [ ]:
delta , device , (position_history, ground, charging)  = run_benchmark_scenario(scenario, SensorPlacementOptimization, RandomDroneRoutingStrategy, custom_initialization_parameters = {"burnmap_filename": "./WideDataset/0001/burn_map.npy"}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
create_scenario_video(scenario[:len(position_history)],drone_locations_history=position_history,starting_time=0,out_filename='test_simulation', ground_sensor_locations = ground, charging_stations_locations = charging)

Instead of running a benchmark on a single scenario, we are interested in running the benchmark on all scenarii of a given layout (potentially in parallel!)

1. We use `run_benchmark_scenarii_sequential`

In [ ]:
run_benchmark_scenarii_sequential("MinimalDataset/0001/scenarii/", RandomSensorPlacementStrategy, RandomDroneRoutingStrategy, custom_initialization_parameters_function = get_burnmap_parameters, custom_step_parameters_function = return_no_custom_parameters, starting_time=0, max_n_scenarii=3)
run_benchmark_scenarii_sequential("MinimalDataset/0001/Satellite_Images_Mask/", RandomSensorPlacementStrategy, RandomDroneRoutingStrategy, custom_initialization_parameters_function = get_burnmap_parameters, custom_step_parameters_function = return_no_custom_parameters, starting_time=0, max_n_scenarii=3, file_format="jpg")

On all of the dataset!

In [ ]:
benchmark_on_sim2real_dataset("WideDataset/", SensorPlacementStrategy, DroneRoutingStrategy, custom_initialization_parameters_function = lambda x: None, custom_step_parameters_function = return_no_custom_parameters, max_n_scenarii=100, starting_time=0)

In [ ]:
# That's very fast to run!
time_start = time.time()
scenario = load_scenario_npy("WideDataset/0001/scenarii/0001_00002.npy")
device, delta_t, _ = run_benchmark_scenario(scenario, RandomSensorPlacementStrategy, DroneRoutingOptimizationSlow, custom_initialization_parameters = {"burnmap_filename": "./WideDataset/0001/burn_map.npy", "call_every_n_steps": 5, "optimization_horizon": 10}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
print("Fire detected in ", delta_t, "time steps by device: ", device)
print(f"Time taken to run benchmark on the scenario: {time.time() - time_start} seconds")


In [ ]:
# define function to get parameters for optimal drone routing strategy
# def get_routing_optimization_parameters(input_dir:str):
#     return {
#         "burnmap_filename": f"{input_dir}/burn_map.npy",
#         "call_every_n_steps": 5,
#         "optimization_horizon": 10
#     }

# add the log
